In [1]:
!unzip colabdataset.zip

Archive:  colabdataset.zip
   creating: images/
  inflating: images/2018-06-01.tiff  
  inflating: images/2019-05-10.tiff  
  inflating: images/2019-05-17.tiff  
  inflating: images/2019-05-28.tiff  
  inflating: images/2020-05-02.tiff  
  inflating: images/2020-05-09.tiff  
  inflating: images/2020-05-31.tiff  
  inflating: images/2021-05-02.tiff  
  inflating: images/2021-05-04.tiff  
  inflating: images/2021-05-15.tiff  
  inflating: images/2021-05-26.tiff  
  inflating: images/2021-06-03.tiff  
  inflating: images/2021-06-03_2.tiff  
  inflating: images/2021-06-03_3.tiff  
  inflating: images/2021-06-03_4.tiff  
  inflating: images/2021-06-03_5.tiff  
  inflating: images/2021-06-06.tiff  
  inflating: images/2021-06-06_2.tiff  
  inflating: images/2021-06-08.tiff  
  inflating: images/2021-06-16.tiff  
  inflating: images/2021-06-16_2.tiff  
   creating: weather/
  inflating: weather/2018-06-01_weather.csv  
  inflating: weather/2019-05-10_weather.csv  
  inflating: weather/2019-05

In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 52.2 MB/s eta 0:00:00


# Предобработка изображений и таблиц
Масштабирование, заполнение пропущенных значений, приведение к одной размерности

In [119]:
import rasterio
import pandas as pd
import numpy as np
import os

# Путь к папке с изображениями и таблицами
image_folder = './images'
csv_folder = './weather'

# Загрузка изображений и масок
images = []
masks = []
for filename in os.listdir(image_folder):
    if filename.endswith('.tiff'):
        with rasterio.open(os.path.join(image_folder, filename)) as src:
            img = src.read()  # Чтение всех каналов

            images.append(img[:4])
            # Предполагается, что маска хранится в последнем канале
            masks.append(img[-1])  # Последний канал как маска

# Загрузка погодных данных
weather_data = {}
for filename in os.listdir(csv_folder):
    if filename.endswith('.csv'):
        data = pd.read_csv(os.path.join(csv_folder, filename))
        data = data.drop(columns=['time', 'Порывы ветра'])

        # Заполнение пропущенных значений средними значениями по столбцам
        data = data.fillna(data.mean())
        data = data.fillna(0)
        weather_data[filename] = data

# Приведение изображений к одному размеру
from skimage.transform import resize

target_size = (256, 256)  # Пример целевого размера
processed_images = []
processed_masks = []

for img, mask in zip(images, masks):
    img_resized = resize(img.transpose(1, 2, 0), target_size, anti_aliasing=True)
    mask_resized = resize(mask, target_size, order=0)  # Маска без интерполяции
    processed_images.append(img_resized)
    processed_masks.append(mask_resized)

# Нормализация значений пикселей
processed_images = np.array(processed_images) / 255.0
processed_masks = np.array(processed_masks)  # Маски могут быть целыми числами

# Преобразование масок в формат one-hot
num_classes = 5  # Укажите количество классов
processed_masks_one_hot = np.eye(num_classes)[processed_masks.astype(int)]

# Обработка погодных данных
weather_features = []
for filename, data in weather_data.items():
    features = data[['Тсред', 'Тмин', 'Тмакс', 'Осадки всего', 'Направление ветра', 'Скорость ветра', 'Давление на уровне моря']].values
    weather_features.append(features)

# Приведение к одинаковой размерности
max_length = max(len(f) for f in weather_features)
for i in range(len(weather_features)):
    if len(weather_features[i]) < max_length:
        weather_features[i] = np.pad(weather_features[i], ((0, max_length - len(weather_features[i])), (0, 0)), 'constant')

weather_features = np.array(weather_features)

# Архитектура модель
U-NET + конкатенация с табличными данными

In [120]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Flatten, Dense, Reshape
from tensorflow.keras.optimizers import Adam

# Входные слои
image_input = Input(shape=(256, 256, 4))  # 4 канала - r, g, b, ik
weather_input = Input(shape=(max_length, 7))  # 7 признаков о погоде

# Кодировщик U-NET
c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
p1 = MaxPooling2D((2, 2))(c1)

c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
p2 = MaxPooling2D((2, 2))(c2)

c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
p3 = MaxPooling2D((2, 2))(c3)

# Декодировщик U-NET
u1 = UpSampling2D((2, 2))(p3)
c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
m1 = Concatenate()([c4, c3])

u2 = UpSampling2D((2, 2))(m1)
c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
m2 = Concatenate()([c5, c2])

# Выходной слой
output = Conv2D(num_classes, (1, 1), activation='softmax')(m2)

# Обработка табличных данных
y = Flatten()(weather_input)

# Объединение признаков
combined = Concatenate()([Flatten()(output), y])
z = Dense(128, activation='relu')(combined)
z = Dense(256 * 256 * num_classes, activation='sigmoid')(z)
final_output = Reshape((256, 256, num_classes))(z)

model = Model(inputs=[image_input, weather_input], outputs=final_output)

# Использование categorical_crossentropy
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Обучение модели

In [121]:
from sklearn.model_selection import train_test_split

X_images_train, X_images_test, y_masks_train, y_masks_test, X_weather_train, X_weather_test = train_test_split(
    processed_images,
    processed_masks_one_hot,
    weather_features,
    test_size=0.2,  # 20% данных для теста
    random_state=42  # Для воспроизводимости
)

model.fit([X_images_train, X_weather_train], y_masks_train, epochs=10, batch_size=8, validation_split=0.2)

Epoch 1/10
2/2 [==============================] - 3s 807ms/step - loss: 1.6146 - accuracy: 0.2330 - val_loss: 1.4985 - val_accuracy: 0.4041
Epoch 2/10
2/2 [==============================] - 1s 620ms/step - loss: 1.4571 - accuracy: 0.4633 - val_loss: 1.2839 - val_accuracy: 0.7037
Epoch 3/10
2/2 [==============================] - 2s 669ms/step - loss: 1.1988 - accuracy: 0.7417 - val_loss: 1.0141 - val_accuracy: 0.8624
Epoch 4/10
2/2 [==============================] - 2s 677ms/step - loss: 0.9312 - accuracy: 0.8746 - val_loss: 0.8002 - val_accuracy: 0.9019
Epoch 5/10
2/2 [==============================] - 2s 691ms/step - loss: 0.6887 - accuracy: 0.9007 - val_loss: 0.6948 - val_accuracy: 0.9114
Epoch 6/10
2/2 [==============================] - 2s 678ms/step - loss: 0.5515 - accuracy: 0.9071 - val_loss: 0.6634 - val_accuracy: 0.9145
Epoch 7/10
2/2 [==============================] - 2s 676ms/step - loss: 0.4685 - accuracy: 0.9084 - val_loss: 0.6619 - val_accuracy: 0.9160
Epoch 8/10
2/2 [====

Оценка модели

In [115]:
loss, accuracy = model.evaluate([X_images_test, X_weather_test], y_masks_test)

1/1 [==============================] - 0s 112ms/step - loss: 0.1266 - accuracy: 0.9870


# Сохранение модели

In [97]:
model.save('./unet-tabular_mm_v2')

In [98]:
import shutil

source_dir = './unet-tabular_mm_v2'
archive_path = './unet-tabular_mm_v2'

shutil.make_archive(archive_path, 'zip', source_dir, verbose=True)

'/content/unet-tabular_mm_v2.zip'

# Предобработка файлов и предсказание маски

In [116]:
# Путь к файлам
image_path = './test.tiff'
csv_path = './test_weather.csv'

# Загрузка изображения
with rasterio.open(image_path) as src:
    img = src.read()  # Чтение всех каналов
    img = img.transpose(1, 2, 0)  # Перестановка осей для совместимости с skimage

# Приведение изображения к одному размеру
target_size = (256, 256)  # Пример целевого размера
img_resized = resize(img, target_size, anti_aliasing=True)

# Нормализация значений пикселей
img_resized = img_resized / 255.0

# Загрузка погодных данных
weather_data = pd.read_csv(csv_path, sep=";")
weather_data = weather_data.drop(columns=['time', 'Порывы ветра'])
weather_data = weather_data.fillna(weather_data.mean())
weather_data = weather_data.fillna(0)

# Выбор необходимых признаков
weather_features = weather_data[['Тсред', 'Тмин', 'Тмакс', 'Осадки всего', 'Направление ветра', 'Скорость ветра', 'Давление на уровне моря']].values

# Преобразование в формат, совместимый с моделью
weather_features = np.pad(weather_features, ((0, 0), (0, 0)), 'constant')

# Создание входных данных для модели
image_input = np.expand_dims(img_resized, axis=0)  # Добавление размерности для батча
weather_input = np.expand_dims(weather_features, axis=0)  # Добавление размерности для батча

# Предсказание
prediction = model.predict([image_input, weather_input])

# Удаление лишней оси из предсказанной маски
predicted_mask = np.argmax(prediction, axis=-1)  # Удаление первой оси и получение индексов классов

resized_pred = resize(predicted_mask.squeeze(), (316, 516), order=0)

# Преобразование в бинарную маску (если необходимо)
pred_mask = (resized_pred > 0).astype(np.uint8)

1/1 [==============================] - 0s 175ms/step


Сохранение предсказания

In [117]:
df = pd.DataFrame(pred_mask)
df.to_csv("submit.csv", sep=',', index=False, header=False)